# **Importing Libraries**

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
import plotly.express as px
import plotly.figure_factory as ff
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
!pip install pywaffle
from pywaffle import Waffle
from sklearn import tree
import sklearn.metrics
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, confusion_matrix
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
import lightgbm as lgb
from sklearn.model_selection import train_test_split
import warnings
from yellowbrick.classifier import ClassificationReport
from yellowbrick.classifier import ConfusionMatrix
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, recall_score, roc_auc_score, precision_score, f1_score
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
warnings.filterwarnings("ignore")


#  **Reading data**

In [ ]:
df=pd.read_csv("../input/heart-attack-analysis-prediction-dataset/heart.csv")
df

# **Dataset Description**

Age : Age of the patient

Sex : Sex of the patient

exang: exercise induced angina (1 = yes; 0 = no)

ca: number of major vessels (0-3)

cp : Chest Pain type chest pain type

Value 1: typical angina Value 2: atypical angina Value 3: non-anginal pain Value 4: asymptomatic trtbps : resting blood pressure (in mm Hg)

chol : cholestoral in mg/dl fetched via BMI sensor

fbs : (fasting blood sugar > 120 mg/dl) (1 = true; 0 = false)

rest_ecg : resting electrocardiographic results

Value 0: normal Value 1: having ST-T wave abnormality (T wave inversions and/or ST elevation or depression of > 0.05 mV) Value 2: showing probable or definite left ventricular hypertrophy by Estes' criteria thalach : maximum heart rate achieved

target : 0= less chance of heart attack 1= more chance of heart attack

# Data Distribution

In [ ]:
heart_disease = df['output'].value_counts().values[0]
heart_disease_no =  df['output'].value_counts().values[1]

no_percent = int(round(heart_disease_no/(heart_disease+heart_disease_no) * 100, 0))
yes_percent = int(round(heart_disease/(heart_disease+heart_disease_no)* 100, 0))

fig = plt.figure(FigureClass = Waffle, 
                 constrained_layout = True,
                 figsize = (10,9),
                 facecolor = '#f6f5f5',dpi = 100,
                 
                 plots = {'121':
                          {     
                           'rows':7,
                           'columns': 7,
                           'values' : [yes_percent],
                            'colors' : ['#fe346e'],
                              'vertical' : True,
                              'interval_ratio_x': 0.005,
                              'interval_ratio_y': 0.005,
                              'icons' : 'heart',
                              'icon_legend': False,
                              'icon_size':20,
                              'plot_anchor':'C',
                              'alpha':0.8,
                              'starting_location': 'NE'
                          },
                          
                          '122' :
                          { 
                            'rows': 7,
                            'columns':7,
                            'values':[no_percent],         
                              'colors' : ['#512b58'],
                              'vertical': True,
                              'interval_ratio_x': 0.005,
                              'interval_ratio_y':0.005,
                              'icons' : 'heart',
                              'icon_legend' :False,
                              'icon_size':20,
                              'plot_anchor':'C',
                              'alpha':0.8,
                              'starting_location': 'NE'
                                                      
                           }
                         },
                   
)
fig.text(0.12, 0.68, 'Heart Attack ({}%)'.format(yes_percent), {'font':'Serif', 'size':20,'weight':'bold' ,'color':'black'},alpha = 0.5,)
fig.text(0.55, 0.68, "No Heart Attack({}%)".format(no_percent), {'font':'Serif', 'size':20,'weight':'bold', 'color':'black'}, alpha = 0.5)


fig.show()

As 54% of the person have heart attack so data is almost balanced so no need to balance the data.



# Distribution of Age w.r.t Heart Disease 

In [ ]:
fig = px.histogram(df, x="age",color="output",
                   marginal="box",
                   hover_data=df.columns,
                  color_discrete_sequence=['#512b58','#f64c72'])
fig.show()

People having less age has higher chance of heart attack

# Distribution of Gender w.r.t Heart Disease 

In [ ]:
name='sex'
l = [(100)*df[df[name]==0]['output'].sum()/len(df[df[name]==0]['output']),
     (100)*df[df[name]==1]['output'].sum()/len(df[df[name]==1]['output'])]

fig = plt.figure(FigureClass = Waffle, 
                 constrained_layout = True,
                 figsize = (10,8),
                 facecolor = '#f6f5f5',dpi = 100,
                 
                 plots = {'121':
                          {     
                           'rows':7,
                           'columns': 7,
                           'values' : [l[0],100-l[0]],
                            'colors' : ['#fe346e','#512b58'],
                              'vertical' : True,
                              'interval_ratio_x': 0.005,
                              'interval_ratio_y': 0.005,
                              'icons' : 'male',
                              'icon_legend': False,
                              'icon_size':20,
                              'plot_anchor':'C',
                              'alpha':0.8,
                              'starting_location': 'NE'
                          },
                          
                          '122' :
                          { 
                            'rows': 7,
                            'columns':7,
                            'values':[l[1],100-l[1]],         
                              'colors' : ['#fe346e','#512b58'],
                              'vertical': True,
                              'interval_ratio_x': 0.005,
                              'interval_ratio_y':0.005,
                              'icons' : 'female',
                              'icon_legend' :False,
                              'icon_size':20,
                              'plot_anchor':'C',
                              'alpha':0.8,
                              'starting_location': 'NE'
                                                      
                           }
                         },
                   
)
fig.text(0.25, 0.24, '{}%'.format(l[0]), {'font':'Serif', 'size':16,'weight':'bold' ,'color':'#2c003e'},alpha = 1,)
fig.text(0.6, 0.24, '{}%'.format(l[1]), {'font':'Serif', 'size':16,'weight':'bold', 'color':'#2c003e'}, alpha = 1)
fig.text(0.95,0.55, 'Heart Attack ', {'font': 'Serif','weight':'bold','Size': '16','weight':'bold','style':'normal', 'color':'#fe346e'})
fig.text(0.95,0.65, 'No Heart Attack', {'font': 'Serif','weight':'bold', 'Size': '16','style':'normal', 'weight':'bold','color':'#512b58'},alpha = 1)

75% of male suffers from heart attack whereas only 44% of female suffer from it which males are more probable to suffer from heart attack.

# Distribution of Blood pressure and cholestrol w.r.t Heart Disease

In [ ]:
data=df
fig=plt.figure(figsize=(14, 6))
fig = plt.subplot(121)
sns.distplot(data[data['output'] == 0]["chol"], color='#512b58',label='No heart Disease') 
sns.distplot(data[data['output'] == 1]["chol"], color='#fe346e',label='Heart Disease') #Red for heart disease
plt.title('Heart Attack distibution over Cholestrol ', fontsize=15)
plt.legend()
# plt.show()
fig = plt.subplot(122)
sns.distplot(data[data['output'] == 0]["trtbps"], color='#512b58',label='No heart Disease') 
sns.distplot(data[data['output'] == 1]["trtbps"], color='#fe346e',label='Heart Disease') #Red for heart disease
plt.title('Heart Attack distibution over Blood pressure ', fontsize=15)
plt.legend()

People having cholestrol 120-250 and blood between 110 to 140 are more likely to have heart attack.

# Distribution of Chest Pain w.r.t Heart Disease

In [ ]:
name='cp'
l = [(100)*df[df[name]==0]['output'].sum()/len(df[df[name]==0]['output']),
     (100)*df[df[name]==1]['output'].sum()/len(df[df[name]==1]['output']), 
     (100)*df[df[name]==2]['output'].sum()/len(df[df[name]==2]['output']),
     (100)*df[df[name]==3]['output'].sum()/len(df[df[name]==3]['output'])]

fig = plt.figure(FigureClass = Waffle, 
                 constrained_layout = True,
                 figsize = (10,8),
                 facecolor = '#f6f5f5',dpi = 100,
                 
                 plots = {'141':
                          {     
                           'rows':7,
                           'columns': 7,
                           'values' : [l[0],100-l[0]],
                            'colors' : ['#fe346e','#512b58'],
                              'vertical' : True,
                              'interval_ratio_x': 0.005,
                              'interval_ratio_y': 0.005,
                              'icons' : 'heart',
                              'icon_legend': False,
                              'icon_size':16,
                              'plot_anchor':'C',
                              'alpha':0.8,
                              'starting_location': 'NE'
                          },
                          
                          '142' :
                          { 
                            'rows': 7,
                            'columns':7,
                            'values':[l[1],100-l[1]],         
                              'colors' : ['#fe346e','#512b58'],
                              'vertical': True,
                              'interval_ratio_x': 0.005,
                              'interval_ratio_y':0.005,
                              'icons' : 'heart',
                              'icon_legend' :False,
                              'icon_size':16,
                              'plot_anchor':'C',
                              'alpha':0.8,
                              'starting_location': 'NE'
                                                      
                           },
                          '143' :
                          { 
                            'rows': 7,
                            'columns':7,
                            'values':[l[2],100-l[2]],         
                              'colors' : ['#fe346e','#512b58'],
                              'vertical': True,
                              'interval_ratio_x': 0.005,
                              'interval_ratio_y':0.005,
                              'icons' : 'heart',
                              'icon_legend' :False,
                              'icon_size':16,
                              'plot_anchor':'C',
                              'alpha':0.8,
                              'starting_location': 'NE'
                                                      
                           },
                          '144' :
                          { 
                            'rows': 7,
                            'columns':7,
                            'values':[l[3],100-l[3]],         
                              'colors' : ['#fe346e','#512b58'],
                              'vertical': True,
                              'interval_ratio_x': 0.005,
                              'interval_ratio_y':0.005,
                              'icons' : 'heart',
                              'icon_legend' :False,
                              'icon_size':16,
                              'plot_anchor':'C',
                              'alpha':0.8,
                              'starting_location': 'NE'
                                                      
                           }
                          
                         },
                   
)
fig.text(0.15, 0.36, '{:.2f}%'.format(l[0]), {'font':'Serif', 'size':16,'weight':'bold' ,'color':'#2c003e'},alpha = 1,)
fig.text(0.38, 0.36, '{:.2f}%'.format(l[1]), {'font':'Serif', 'size':16,'weight':'bold', 'color':'#2c003e'}, alpha = 1)
fig.text(0.6, 0.36, '{:.2f}%'.format(l[2]), {'font':'Serif', 'size':16,'weight':'bold' ,'color':'#2c003e'},alpha = 1,)
fig.text(0.78, 0.36, '{:.2f}%'.format(l[3]), {'font':'Serif', 'size':16,'weight':'bold', 'color':'#2c003e'}, alpha = 1)
fig.text(0.95,0.55, 'Heart Attack ', {'font': 'Serif','weight':'bold','Size': '16','weight':'bold','style':'normal', 'color':'#fe346e'})
fig.text(0.95,0.60, 'No Heart Attack', {'font': 'Serif','weight':'bold', 'Size': '16','style':'normal', 'weight':'bold','color':'#512b58'},alpha = 1)


Peaople having high chest pain are highly probable to suffer from heart disease.

# Disrtribution of one feature w.r.t other with hue = Heart Disease

In [ ]:
fig = plt.figure(figsize=(24,12), dpi = 100)
gs = fig.add_gridspec(3,2)
gs.update(wspace=0.15, hspace=0.3)

ax0 = fig.add_subplot(gs[0,0])
ax1 = fig.add_subplot(gs[0,1])
ax2 = fig.add_subplot(gs[1,0])
ax3 = fig.add_subplot(gs[1,1])
ax4 = fig.add_subplot(gs[2,0])
ax5 = fig.add_subplot(gs[2,1])

fig.patch.set_facecolor('#f6f5f5')
ax0.set_facecolor('#f6f5f5')
ax1.set_facecolor('#f6f5f5')
ax2.set_facecolor('#f6f5f5')
ax3.set_facecolor('#f6f5f5')
ax4.set_facecolor('#f6f5f5')
ax5.set_facecolor('#f6f5f5')


# ever_married, gender, residence, heart_disease and work_type

healthy = df[df['output']==0]
heart_attack= df[df['output']==1]

gender_order = [0,1]
chest_pain = [0,1,2,3]
thall = [0,1,2,3]
caa = [0,1,2,3]

col1 = ["#4b4b4c","#fe346e"]
colormap1 = matplotlib.colors.LinearSegmentedColormap.from_list("", col1, N = 256)
col2 = ["#4b4b4c","#512b58"]
colormap2 = matplotlib.colors.LinearSegmentedColormap.from_list("", col2)

heart_attack = pd.crosstab(heart_attack['sex'],[heart_attack['cp']],normalize='index').loc[gender_order,chest_pain]
no_heart_attack = pd.crosstab(healthy['sex'],[healthy['cp']], normalize='index').loc[gender_order,chest_pain]

sns.heatmap(ax=ax0, data=heart_attack, linewidths= 0,
            square=False, cbar_kws={"orientation": "horizontal"}, cbar=False,linewidth=3, cmap = col1,annot=True, fmt='1.0%',annot_kws={"fontsize":14}, alpha = 0.9)

sns.heatmap(ax=ax1, data=no_heart_attack, linewidths=0, 
            square=False, cbar_kws={"orientation": "horizontal"}, cbar=False,linewidth=3, cmap = col2,annot=True, fmt='1.0%',annot_kws={"fontsize":14}, alpha = 0.9)

healthy = df[df['output']==0]
heart_attack= df[df['output']==1]                  
heart_attack = pd.crosstab(heart_attack['sex'],[heart_attack['thall']],normalize='index').loc[gender_order,thall]
no_heart_attack = pd.crosstab(healthy['sex'],[healthy['thall']], normalize='index').loc[gender_order,thall]
sns.heatmap(ax=ax2, data=heart_attack, linewidths= 0,
            square=False, cbar_kws={"orientation": "horizontal"}, cbar=False,linewidth=3, cmap = col1,annot=True, fmt='1.0%',annot_kws={"fontsize":14}, alpha = 0.9)

sns.heatmap(ax=ax3, data=no_heart_attack, linewidths=0, 
            square=False, cbar_kws={"orientation": "horizontal"}, cbar=False,linewidth=3, cmap = col2,annot=True, fmt='1.0%',annot_kws={"fontsize":14}, alpha = 0.9)

healthy = df[df['output']==0]
heart_attack= df[df['output']==1]                  
heart_attack = pd.crosstab(heart_attack['sex'],[heart_attack['caa']],normalize='index').loc[gender_order,thall]
no_heart_attack = pd.crosstab(healthy['sex'],[healthy['caa']], normalize='index').loc[gender_order,thall]
sns.heatmap(ax=ax4, data=heart_attack, linewidths= 0,
            square=False, cbar_kws={"orientation": "horizontal"}, cbar=False,linewidth=3, cmap = col1,annot=True, fmt='1.0%',annot_kws={"fontsize":14}, alpha = 0.9)

sns.heatmap(ax=ax5, data=no_heart_attack, linewidths=0, 
            square=False, cbar_kws={"orientation": "horizontal"}, cbar=False,linewidth=3, cmap = col2,annot=True, fmt='1.0%',annot_kws={"fontsize":14}, alpha = 0.9)


ax0.text(0,-0.1,'Heart Attack Pecentage ', {'font':'serif', 'color':"#fe346e", 'size':30},alpha = 0.9)
ax1.text(0,-0.1,'No Heart Attack Percentage', {'font':'serif', 'color':"#512b58", 'size':30}, alpha =0.9)

ax0.axes.set_xticklabels(['cp-0', 'cp-1','cp-2', 'cp-3'], {'font':'serif', 'color':'black', 'size':16})
ax1.axes.set_xticklabels(['cp-0', 'cp-1','cp-2', 'cp-3'], {'font':'serif', 'color':'black', 'size':16})
ax2.axes.set_xticklabels(['thall-0','thall-1','thall-2','thall-3'], {'font':'serif', 'color':'black', 'size':16})
ax3.axes.set_xticklabels(['thall-0','thall-1','thall-2','thall-3'], {'font':'serif', 'color':'black', 'size':16})
ax4.axes.set_xticklabels(['caa-0','caa-1','caa-2','caa-3'], {'font':'serif', 'color':'black', 'size':16})
ax5.axes.set_xticklabels(['caa-0','caa-1','caa-2','caa-3'], {'font':'serif', 'color':'black', 'size':16})

ax0.axes.set_yticklabels(['Male','Female'], {'font':'serif', 'color':'black', 'size':16}, rotation = 0)
ax2.axes.set_yticklabels(['Male','Female'], {'font':'serif', 'color':'black', 'size':16}, rotation = 0)
ax4.axes.set_yticklabels(['Male','Female'], {'font':'serif', 'color':'black', 'size':16}, rotation = 0)

ax0.set_xlabel('')
ax0.set_ylabel('')
ax1.set_xlabel('')
ax1.set_ylabel('')
ax2.set_xlabel('')
ax2.set_ylabel('')
ax3.set_xlabel('')
ax3.set_ylabel('')
ax4.set_xlabel('')
ax4.set_ylabel('')
ax5.set_xlabel('')
ax5.set_ylabel('')
ax1.axes.get_yaxis().set_visible(False)
ax3.axes.get_yaxis().set_visible(False)
ax5.axes.get_yaxis().set_visible(False)
fig.show()

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure(figsize=(6,6))
ax = Axes3D(fig)
ax.scatter(data["output"], data["thalachh"],data["age"],  c="red", s=20, alpha=0.5)
plt.xlabel('Heart Attack')
plt.ylabel('Heart Rate')
plt.show()


In [ ]:
fig = plt.figure(figsize=(24,8), dpi = 100)
gs = fig.add_gridspec(2,2)
gs.update(wspace=0.15, hspace=0.3)

ax0 = fig.add_subplot(gs[0,0])
ax1 = fig.add_subplot(gs[0,1])
ax2 = fig.add_subplot(gs[1,0])
ax3 = fig.add_subplot(gs[1,1])


fig.patch.set_facecolor('#f6f5f5')
ax0.set_facecolor('#f6f5f5')
ax1.set_facecolor('#f6f5f5')
ax2.set_facecolor('#f6f5f5')
ax3.set_facecolor('#f6f5f5')



# ever_married, gender, residence, heart_disease and work_type

healthy = df[df['output']==0]
heart_attack= df[df['output']==1]

gender_order = [0,1]
chest_pain = [0,1]
thall = [0,1]
caa = [0,1]

col1 = ["#4b4b4c","#fe346e"]
colormap1 = matplotlib.colors.LinearSegmentedColormap.from_list("", col1, N = 256)
col2 = ["#4b4b4c","#512b58"]
colormap2 = matplotlib.colors.LinearSegmentedColormap.from_list("", col2)

heart_attack = pd.crosstab(heart_attack['sex'],[heart_attack['fbs']],normalize='index').loc[gender_order,chest_pain]
no_heart_attack = pd.crosstab(healthy['sex'],[healthy['fbs']], normalize='index').loc[gender_order,chest_pain]

sns.heatmap(ax=ax0, data=heart_attack, linewidths= 0,
            square=False, cbar_kws={"orientation": "horizontal"}, cbar=False,linewidth=3, cmap = col1,annot=True, fmt='1.0%',annot_kws={"fontsize":14}, alpha = 0.9)

sns.heatmap(ax=ax1, data=no_heart_attack, linewidths=0, 
            square=False, cbar_kws={"orientation": "horizontal"}, cbar=False,linewidth=3, cmap = col2,annot=True, fmt='1.0%',annot_kws={"fontsize":14}, alpha = 0.9)

healthy = df[df['output']==0]
heart_attack= df[df['output']==1]                  
heart_attack = pd.crosstab(heart_attack['sex'],[heart_attack['exng']],normalize='index').loc[gender_order,thall]
no_heart_attack = pd.crosstab(healthy['sex'],[healthy['exng']], normalize='index').loc[gender_order,thall]
sns.heatmap(ax=ax2, data=heart_attack, linewidths= 0,
            square=False, cbar_kws={"orientation": "horizontal"}, cbar=False,linewidth=3, cmap = col1,annot=True, fmt='1.0%',annot_kws={"fontsize":14}, alpha = 0.9)

sns.heatmap(ax=ax3, data=no_heart_attack, linewidths=0, 
            square=False, cbar_kws={"orientation": "horizontal"}, cbar=False,linewidth=3, cmap = col2,annot=True, fmt='1.0%',annot_kws={"fontsize":14}, alpha = 0.9)


ax0.text(0,-0.1,'Heart Attack Pecentage ', {'font':'serif', 'color':"#fe346e", 'size':30},alpha = 0.9)
ax1.text(0,-0.1,'No Heart Attack Percentage', {'font':'serif', 'color':"#512b58", 'size':30}, alpha =0.9)

ax0.axes.set_xticklabels(['No blood sugar', 'Blood sugar'], {'font':'serif', 'color':'black', 'size':16})
ax1.axes.set_xticklabels(['No blood sugar', 'Blood sugar'], {'font':'serif', 'color':'black', 'size':16})
ax2.axes.set_xticklabels(['NO induced engina','Induced engina'], {'font':'serif', 'color':'black', 'size':16})
ax3.axes.set_xticklabels(['NO induced engina','Induced engina'], {'font':'serif', 'color':'black', 'size':16})
ax0.axes.set_yticklabels(['Male','Female'], {'font':'serif', 'color':'black', 'size':16}, rotation = 0)
ax2.axes.set_yticklabels(['Male','Female'], {'font':'serif', 'color':'black', 'size':16}, rotation = 0)

ax0.set_xlabel('')
ax0.set_ylabel('')
ax1.set_xlabel('')
ax1.set_ylabel('')
ax2.set_xlabel('')
ax2.set_ylabel('')
ax3.set_xlabel('')
ax3.set_ylabel('')

ax1.axes.get_yaxis().set_visible(False)
ax3.axes.get_yaxis().set_visible(False)

fig.show()

Conclusion :

73% of male suffers from heart attack whereas only 45% of female suffer from heart attack so males are more likely to suffer from it.

People having chest pain are highly probably to suffer from heart attack.

Blood sugar doesnot have that much impact on heart attack i.e. person having heart attack is independent of whether person have blood sugar or not.

Person doesnot having exercied indeced enginal are higly probable to suffer from heart attack.

People having low exercise induced angina are more probable to suffer from heart disease although age doesnot contribute as much to heart attack.

# Correlation Among Attributes

In [ ]:
plt.figure(figsize=(15,8))
sns.heatmap(data.corr(),cmap="Blues")

As we can see from the above graph and table, heart attack have positive correlation with chest pain,heart rateand slope whereas have negative correlation age, induced engina and major vessels.

# Data Preprocessing And Predictions

In [ ]:
df.info()

It doesnot contains any null value or categorical data. Along with it data is also balanced from distribution of data.
I will store the output variable and will drop it from data and will divide it into train and test.

In [ ]:
y=df['output']
data = df.drop('output', axis=1)
data.head()

X_train, X_test, y_train, y_test = train_test_split(data, y, test_size=0.20, random_state=0)

In [ ]:
model = tree.DecisionTreeClassifier(max_depth=15)
clf = model.fit(X_train, y_train)
y_predict=clf.predict(X_train)
train_accuracy = sklearn.metrics.accuracy_score(y_predict, y_train)
y_valid=clf.predict(X_test)
y_dt=y_valid
test_accuracy = sklearn.metrics.accuracy_score(y_valid, y_test)

fig=plt.figure(figsize=(12,5))
fig = plt.subplot(121)
plt.title("Confusion metric")
sns.heatmap(confusion_matrix(y_test, y_valid), annot = True)
fig = plt.subplot(122)
classes = ["No Heart Attack","Heart Attack"]
visualizer = ClassificationReport(model, classes=classes, support=True)
visualizer.fit(X_train, y_train)       
visualizer.score(X_test, y_test)        
visualizer.show()  

print("Training accuracy",train_accuracy)
print("Test accuracy",test_accuracy)

In [ ]:
model = GaussianNB()
clf = model.fit(X_train, y_train)
y_predict=clf.predict(X_train)
train_accuracy = sklearn.metrics.accuracy_score(y_predict, y_train)
y_valid=clf.predict(X_test)
y_nb=y_valid
test_accuracy = sklearn.metrics.accuracy_score(y_valid, y_test)

fig=plt.figure(figsize=(12,5))
fig = plt.subplot(121)
plt.title("Confusion metric")
sns.heatmap(confusion_matrix(y_test, y_valid), annot = True)
fig = plt.subplot(122)
classes = ["No Heart Attack","Heart Attack"]
visualizer = ClassificationReport(model, classes=classes, support=True)
visualizer.fit(X_train, y_train)       
visualizer.score(X_test, y_test)        
visualizer.show()  

print("Training accuracy",train_accuracy)
print("Test accuracy",test_accuracy)

In [ ]:
model = XGBClassifier()
clf = model.fit(X_train, y_train)
y_predict=clf.predict(X_train)
train_accuracy = sklearn.metrics.accuracy_score(y_predict, y_train)
y_valid=clf.predict(X_test)
y_xg=y_valid
test_accuracy = sklearn.metrics.accuracy_score(y_valid, y_test)

fig=plt.figure(figsize=(12,5))
fig = plt.subplot(121)
plt.title("Confusion metric")
sns.heatmap(confusion_matrix(y_test, y_valid), annot = True)
fig = plt.subplot(122)
classes = ["No Heart Attack","Heart Attack"]
visualizer = ClassificationReport(model, classes=classes, support=True)
visualizer.fit(X_train, y_train)       
visualizer.score(X_test, y_test)        
visualizer.show()  

print("Training accuracy",train_accuracy)
print("Test accuracy",test_accuracy)

In [ ]:
model =LogisticRegression()
clf = model.fit(X_train, y_train)
y_predict=clf.predict(X_train)
train_accuracy = sklearn.metrics.accuracy_score(y_predict, y_train)
y_valid=clf.predict(X_test)
y_lr=y_valid
test_accuracy = sklearn.metrics.accuracy_score(y_valid, y_test)

fig=plt.figure(figsize=(12,5))
fig = plt.subplot(121)
plt.title("Confusion metric")
sns.heatmap(confusion_matrix(y_test, y_valid), annot = True)
fig = plt.subplot(122)
classes = ["No Heart Attack","Heart Attack"]
visualizer = ClassificationReport(model, classes=classes, support=True)
visualizer.fit(X_train, y_train)       
visualizer.score(X_test, y_test)        
visualizer.show()  

print("Training accuracy",train_accuracy)
print("Test accuracy",test_accuracy)

In [ ]:
model = RandomForestClassifier(n_estimators=3000)
clf = model.fit(X_train, y_train)
y_predict=clf.predict(X_train)
train_accuracy = sklearn.metrics.accuracy_score(y_predict, y_train)
y_valid=clf.predict(X_test)
y_rf=y_valid
test_accuracy = sklearn.metrics.accuracy_score(y_valid, y_test)

fig=plt.figure(figsize=(12,5))
fig = plt.subplot(121)
plt.title("Confusion metric")
sns.heatmap(confusion_matrix(y_test, y_valid), annot = True)
fig = plt.subplot(122)
classes = ["No Heart Attack","Heart Attack"]
visualizer = ClassificationReport(model, classes=classes, support=True)
visualizer.fit(X_train, y_train)       
visualizer.score(X_test, y_test)        
visualizer.show()  

print("Training accuracy",train_accuracy)
print("Test accuracy",test_accuracy)

In [ ]:
lr_df = pd.DataFrame(data=[f1_score(y_test,y_lr),accuracy_score(y_test, y_lr), recall_score(y_test, y_lr), precision_score(y_test, y_lr), roc_auc_score(y_test, y_lr)], 
             columns=['Logistic Regression'], index=["F1","Accuracy", "Recall", "Precision", "ROC AUC Score"])
rf_df = pd.DataFrame(data=[f1_score(y_test,y_rf),accuracy_score(y_test, y_rf), recall_score(y_test, y_rf),precision_score(y_test, y_rf), roc_auc_score(y_test, y_rf)], 
             columns=['Random Forest Score'],index=["F1","Accuracy", "Recall", "Precision", "ROC AUC Score"])
nb_df = pd.DataFrame(data=[f1_score(y_test,y_nb),accuracy_score(y_test, y_nb), recall_score(y_test, y_nb), precision_score(y_test, y_nb), roc_auc_score(y_test, y_nb)], 
             columns=['Naive Bayes'], index=["F1","Accuracy", "Recall", "Precision", "ROC AUC Score"])
# knn_df = pd.DataFrame(data=[f1_score(y_test,y_knn),accuracy_score(y_test, y_knn), recall_score(y_test, y_knn), precision_score(y_test, y_knn), roc_auc_score(y_test, y_knn)], 
#              columns=['KNN Score'], index=["F1","Accuracy", "Recall", "Precision", "ROC AUC Score"])

xg_df = pd.DataFrame(data=[f1_score(y_test,y_xg),accuracy_score(y_test, y_xg), recall_score(y_test, y_xg), precision_score(y_test, y_xg), roc_auc_score(y_test, y_xg)], 
             columns=['XG Boost'], index=["F1","Accuracy", "Recall", "Precision", "ROC AUC Score"])
dt_df = pd.DataFrame(data=[f1_score(y_test,y_dt),accuracy_score(y_test, y_dt), recall_score(y_test, y_dt), precision_score(y_test, y_dt), roc_auc_score(y_test,y_dt)], 
             columns=['Decision Tree'], index=["F1","Accuracy", "Recall", "Precision", "ROC AUC Score"])


df_models = round(pd.concat([lr_df,rf_df,nb_df,dt_df,xg_df], axis=1),3)
colors = ["bisque","ivory","sandybrown","steelblue","lightsalmon"]
colormap = matplotlib.colors.LinearSegmentedColormap.from_list("", colors)

background_color = "white"

fig = plt.figure(figsize=(18,26)) # create figure
gs = fig.add_gridspec(4, 2)
gs.update(wspace=0.1, hspace=0.5)
ax0 = fig.add_subplot(gs[0, :])

sns.heatmap(df_models.T, cmap=colormap,annot=True,fmt=".1%",vmin=0,vmax=0.95, linewidths=2.5,cbar=False,ax=ax0,annot_kws={"fontsize":16})
fig.patch.set_facecolor(background_color) # figure background color
ax0.set_facecolor(background_color) 

ax0.text(0,-0.5,'Model Comparison',fontsize=20,fontweight='bold',fontfamily='serif')
plt.show()

Among all of our models logistic regression gives us the best result.

References :

1.https://www.kaggle.com/kaamraankhan/heart-attack-eda

2.https://www.kaggle.com/bhuvanchennoju/data-stroytelling-auc-focus-on-strokes